### podcast api part

In [ ]:
from listennotes import podcast_api
import pickle
API_KEY = '06673b55481c47c3ac5636b2eddff44d' 
client = podcast_api.Client(api_key=API_KEY)
INITIAL_NEXT_EPISODE_PUB_DATE = 1696380498000
PODCAST_ID = '626ce14412af499d811d046990b2a34c'

URL_DIR = "C:/Users/ee527/Desktop/Big Data Analytics/final_project/podcast_url/"
TITLE_DIR = "C:/Users/ee527/Desktop/Big Data Analytics/final_project/podcast_title/"

In [ ]:
days = lambda x: x*86400*1000*1
INITIAL_NEXT_EPISODE_PUB_DATE = (days(1))
n = 0

while True:

  INITIAL_NEXT_EPISODE_PUB_DATE = INITIAL_NEXT_EPISODE_PUB_DATE - (days(1))
  response = client.fetch_podcast_by_id(id=PODCAST_ID,
                                      next_episode_pub_date=INITIAL_NEXT_EPISODE_PUB_DATE,
                                      sort='newest_first'
                                      )
  response_json = response.json()
  
  
  # start process episode json here(suppose to have 10 episodes in one loop):
  
  episode_lst = response_json['episodes']
  
  print(len(episode_lst))
  
  audio_lst = []
  audio_ep_num_lst = []
  
  for i in range(len(episode_lst)):
    
    title = episode_lst[i]['title']
    
    audio_url = episode_lst[i]['audio']
    audio_lst.append(audio_url)
    ep_num = title
    audio_ep_num_lst.append(ep_num)
    
  
  # save 10 episodes into a picke chunck
  
  fname = f'{n}.pkl'
  
  with open(URL_DIR + fname, 'wb') as f:
      pickle.dump(audio_lst, f)
      
  with open(TITLE_DIR + fname, 'wb') as f2:
      pickle.dump(audio_ep_num_lst, f2)
      
  
  n+=1

  print('next episode pub date', response_json['next_episode_pub_date'])
  if response_json['next_episode_pub_date'] is None:
    
    print("end fetching podcaast")
    break
  
  else:
    
    INITIAL_NEXT_EPISODE_PUB_DATE = response_json['next_episode_pub_date']


  print()

  foo = [x['title'] for x in response_json['episodes']]
  for i in foo:
    print(i)
    
  print("==========================================================")
  
  


In [ ]:
print(audio_lst)
print(audio_ep_num_lst)

### Whisper part

In [20]:
import torch
import time
import whisper
import os
import pickle
import re

print(torch.__version__)
device = torch.device('cuda')
model = whisper.load_model(name="base", device=device)

URL_DIR = "C:/Users/ee527/Desktop/Big Data Analytics/final_project/podcast_url/"
TITLE_DIR = "C:/Users/ee527/Desktop/Big Data Analytics/final_project/podcast_title/"


2.1.0+cu118


In [25]:
# read pickle file to get podcast url

pkl_number = 5
TRANSCRIBE_DIR = "C:/Users/ee527/Desktop/Big Data Analytics/final_project/podcast_transcript/5/"

with open(URL_DIR + f"{pkl_number}.pkl", 'rb') as f:
        
        audio_url_lst = pickle.load(f)
        
print(audio_url_lst)

with open(TITLE_DIR + f"{pkl_number}.pkl", 'rb') as f:
        
        audio_episode_num_lst = pickle.load(f)
        
print(audio_episode_num_lst)

print()

for i in range(len(audio_url_lst)):
        
    cleaned_title = re.sub(r'[^a-zA-Z0-9\s]', '', audio_episode_num_lst[i])
   
    txt_path = TRANSCRIBE_DIR + cleaned_title + ".txt"
    

    start = time.time()
    result = model.transcribe(audio_url_lst[i])
    end = time.time()

    print(f"{audio_episode_num_lst[i]} runtime:", end-start)
    
    # write into txt file

    with open(txt_path, 'w') as f:
            f.write(result['text'])

['https://www.listennotes.com/e/p/1671ec42ca694a219471bb939e729541/', 'https://www.listennotes.com/e/p/6163d428d0cb42a5b38e720203a12f20/', 'https://www.listennotes.com/e/p/46a32f6f62a649308617d211d0a2af0b/']
['The Manhattan Arraignment', 'The First Indictment', 'Prosecuting Donald Trump: A Primer']

The Manhattan Arraignment runtime: 357.34820318222046
The First Indictment runtime: 127.85262036323547
Prosecuting Donald Trump: A Primer runtime: 262.0648376941681
